In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
import pprint

# PATH = '/content/drive/MyDrive/INF634Project' # Martin
PATH = '/content/drive/MyDrive/Colab Notebooks/INF634Project' # Romain
DATA_RELPATH = './data/how2-dataset'

%cd $PATH
print('Notebook running on', PATH)
print(os.listdir())

In [ ]:
!rm -rf majore/

In [ ]:
!git clone https://github.com/Deathn0t/majore.git majore

In [8]:
import os
import io
import re
import glob
from pprint import pprint, pformat

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from IPython.display import display, Markdown, Latex
from tqdm import tqdm

from scripts.load_audio_v1 import  AudioFeatureDataset
from scripts.load_text import TextDataset
from scripts.load_video import VideoDataset
from scripts.load_multimodal_data import MultimodalDataset
from scripts.position_encoder import PositionalEncoding
from scripts.encoders import AudioEncoder, VideoEncoder, DotProductAttention
from scripts.decoder import MultimodalDecoder

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [4]:
# path_how2 = "/Volumes/LaCie/vision/data/" # Jeremy
path_how2 = "/Volumes/T7/data/" # Romain

video_path = os.path.join(path_how2, "resnext101-action-avgpool-300h", "train.npy")

texts_path = os.path.join(path_how2,"how2-300h-v1/data/train", "text.en")
embeddings_path = os.path.join(path_how2, "how2-release/word_embedding/","cmu_partition.train.vec")

In [5]:
video_dataset = VideoDataset(video_path)
audio_dataset = AudioFeatureDataset(path_how2,"train")
text_dataset = TextDataset(texts_path, embeddings_path)

print("Len Video: ", len(video_dataset))
print("Len Audio: ", len(audio_dataset))
print("Len Text: ", len(text_dataset))

multimodal_dataset = MultimodalDataset(video_dataset, audio_dataset, text_dataset)

print("\nLen Multimodal: ", len(multimodal_dataset))

/Users/romainegele/Documents/lX/2020-2021/s1/INF634-Computer-Vision/project/majore/notebooks/scripts/load_text.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return  np.array(text), np.array(splitted), largest_split


Len Video:  184949
Len Audio:  184949
Len Text:  184949

Len Multimodal:  184949


In [13]:
class Net(nn.Module):
    def __init__(self, vocab_size, text_size=225):
        super().__init__()
        
        
        dropout = 0.2
        nhead = 6
        nlayer_audio = 3
        nlayer_video = 1

        video_dim = 2048
        audio_feature_size = 43
        audio_seq_len = 1895
        tied_output = 96
        down_sampling_factor = 5

        self.audio_encoder = AudioEncoder(
            audio_feature_size,
            audio_seq_len,
            tied_output,
            nhead,
            nlayer_audio,
            d_feedforward=480,
            dropout=dropout,
            down_sampling_factor=down_sampling_factor
        )

        self.video_encoder = VideoEncoder(
            video_dim,
            nhead,
            nlayer_video,
            d_model=480,
            d_feedforward=480,
            dropout=dropout
        )

        self.fusion = DotProductAttention(d_model, 480, 480)
        
        text_dim = 100
        self.text_size = text_size
        n_layer = 4
        d_model = 480
        d_feedforward = 1920
        dropout = 0.2
        nhead = 6

        self.decoder = MultimodalDecoder(
            text_dim, self.text_size, vocab_size, nhead,  n_layer, d_model, d_feedforward, dropout
        )
        
    def forward(self, video, audio, text):
        audio_encoding = self.audio_encoder(audio)
        video_encoding = self.video_encoder(video)
        merge_encoding = self.fusion(audio_encoding, video_encoding, video_encoding)
        mask = self.decoder.generate_square_subsequent_mask(self.text_size)
        decoded = self.decoder(text, merge_encoding, tgt_mask=mask)
        return decoded

In [14]:
word2id= text_dataset.vocab_id_dict
id2word = text_dataset.id_vocab_dict

vocab_size = len(word2id)
net = Net(vocab_size).to(device)

UnboundLocalError: local variable 'd_model' referenced before assignment

In [15]:
def process_batch(batch):
    audio = batch["audio"].float()
    video = batch["video"]["video"].unsqueeze(1).float()
    text_emb = batch["text"]["embedding"].float()
    text_id = batch["text"]["id_embedding"].long()
    
    return video, audio, text_emb, text_id

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss() # includes softmax

loss_hist = []

def train(net):
    global loss_hist
    
    BATCH_SIZE = 10
    EPOCHS = 1
    
    dataloader = DataLoader(multimodal_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    
    loss_hist = []
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch}")
        
        loss_hist_epoch = []
        
        for batch in tqdm(dataloader):
            
            video, audio, text_emb, text_id = process_batch(batch)
            
            audio = audio.to(device)
            video = video.to(device)
            text_emb = text_emb.to(device)
            text_id = text_id.to(device)
            
            net.zero_grad()
            
            output = net(video, audio, text_emb)
             # out is of shape [225, 4, 36756] => now of shape [4, 225, 36756]
            output = torch.transpose(output, 0, 1).reshape(-1, vocab_size)
            
            target = text_id.view(-1).long()
            loss = criterion(output,target)

            loss.backward()
            optimizer.step()
            
            loss_hist_epoch.append(loss.item())
            print(loss_hist_epoch[-1])
            
        loss_hist.append(loss_hist_epoch)
        break

In [16]:
train(net)

  0%|          | 0/18495 [00:00<?, ?it/s]

Epoch 0
torch.Size([379, 10, 480])
torch.Size([379, 10, 480])


  0%|          | 1/18495 [00:12<62:21:06, 12.14s/it]

1.9464263916015625
torch.Size([379, 10, 480])
torch.Size([379, 10, 480])


  0%|          | 2/18495 [00:24<62:14:57, 12.12s/it]

1.3215463161468506
torch.Size([379, 10, 480])
torch.Size([379, 10, 480])


  0%|          | 2/18495 [00:35<91:56:09, 17.90s/it]


KeyboardInterrupt: 